# Data Science at UCSB

# Python for Data Science: ML Crash Course

## Jason Freeberg, Fall 2016 

Okay! Today will be a crash course in machine learning. I'll explain things at a high level and use Sci Kit Learn to show real examples. "Feature engineering" is the creation or collection of predictors for a machine learning pipeline, so we're covering ML first.

To make an oversimplification, let's assume we have some set of *p* predictors, **X<sub>1</sub>, X<sub>2</sub>, X<sub>3</sub> ... X<sub>p</sub> ** for *n* observations. Then we have a corresponding set of dependent *n* variables, **Y**. **X** could be a set of 100 people (n=100), each with 10 variables (p=10) like height, weight, sex, location, education level, and so on. And, in the same example, **Y** could be that person's salary. What I just described is a *regression* problem. Where we have **X** and **Y**, and **Y** is a continuous variable. Now, from **X** and **Y** we can *learn* **F**, the mapping from **X** to **Y**. We can express it as **F**(**X**) = **Y** or in matrix notation as...

$$ F \left(
\begin{matrix}
X_{1,1} & ... & X_{1,p} \\
\vdots & \ddots & \vdots \\
X_{n,1} & ... & X_{n,p} \\
\end{matrix}
\right) 
= 
\begin{bmatrix}
Y_1 \\
\vdots \\
Y_n
\end{bmatrix}
$$

There are two main branches of machine learning...

### Supervised Learning
Like the example above, supervised learning involves using a matrix of *n*X*p* inputs, **X**, and *n* crorresponding outputs, **Y**, to build a statistical model that can then give predicted outputs from new, unseen inputs. As you might expect, this type of learning has broad applications to business, healthcare, and physics.

In these problems, we assume that there is a true, unknown function of the form below, where espison is an irriducible error term with variance zero. We are attempting to approximate **F** as closely as possible because that is the error that we can reduce.

$$ Y = f(X) + \epsilon $$
$$ ... $$
$$ 
\begin{align}
 E(Y - \hat Y)^2 & = E(f(X) + \epsilon - \hat f(X))^2 \\
& = (f(X) - \hat f(X))^2  + Var(\epsilon) \\
 \end{align}$$


#### Sci Kit Lean Example
Similar to the problem above, imagine we have both **X** and **Y** and try to learn the mapping between them. But what if our dependant variable, **Y**, doesn't span the real numbers? There are many casses where we're trying to *classify* our outcomes... good or bad, alive or dead, pay or default... and these are **quantitative** or **classification** problems. 

Moreover, we can have multiple classes in **Y**. Think of tax brackets, image recognition, or types of crime. Food for thought: we can turn a regression problem into a classification problem simply by *binning* our outcomes. Salary in dollars would become income brackets. Then we could use classification algorithms instead.

In [4]:
# Iris is a classic dataset. It holds various measurements of flowers and their species.
# If we want to predict species from the measurements, what kind of problem are we 
# working with?

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation, metrics
from urllib.request import urlopen
import os


def read_csv_from_url(URL, columnNames):
    response = urlopen(URL)
    lines = pd.read_csv(response,
                        header=None,
                        index_col=False)
    dataframe = pd.DataFrame(lines)
    dataframe.columns = columnNames
    return dataframe

# Seeds make our random methods reproducible.
seed = 123

# Load the Iris dataset
irisURL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
irisNames = ['sepalLength', 'sepalWidth', 'petalLength', 'petalWidth', 'class']
iris = read_csv_from_url(irisURL, irisNames)
print('N rows =', iris.shape[0], '\n', 'N cols =', iris.shape[1])
print('Classes in the dependent var. =', set(iris['class']))
print(iris.head())


N rows = 150 
 N cols = 5
Classes in the dependent var. = {'Iris-virginica', 'Iris-versicolor', 'Iris-setosa'}
   sepalLength  sepalWidth  petalLength  petalWidth        class
0          5.1         3.5          1.4         0.2  Iris-setosa
1          4.9         3.0          1.4         0.2  Iris-setosa
2          4.7         3.2          1.3         0.2  Iris-setosa
3          4.6         3.1          1.5         0.2  Iris-setosa
4          5.0         3.6          1.4         0.2  Iris-setosa


#### Training and Test Sets

When you're studying for an exam at school you have your notes, review packets, and maybe some recorded lectures. You study, or *train*, with these materials and then you take the exam to see how well you understand the material you studied. The test has questions *similar* to what you studied, but not equal. So if you studied hard, you should be able to **generalize** well to these new "inputs".

Likewise in machine learning, we need some data with the recorded inputs *and* outputs. Then when a model is trained on that data, we need to evaulate the model's perfomance by giving it new, unseen data. This is a common paradigm in machine and statistical learning, but it can be easy to mess up.

In [7]:
# Split the data into train and test sets
train, test = cross_validation.train_test_split(iris,
                                                test_size=0.3,
                                               random_state=seed)

# Coerce the independent and dependent variable of the training set to NumPy arrays.
predictors = np.array(train.ix[:, 0:-1])
variable = np.array(train.ix[:, -1])

print('Predictors:', '\n', predictors[:5])
print('Dependent variable:', '\n', variable[:5])


Predictors: 
 [[ 5.8  2.8  5.1  2.4]
 [ 6.3  3.4  5.6  2.4]
 [ 5.5  2.3  4.   1.3]
 [ 5.1  3.8  1.5  0.3]
 [ 4.4  3.   1.3  0.2]]
Dependent variable: 
 ['Iris-virginica' 'Iris-virginica' 'Iris-versicolor' 'Iris-setosa'
 'Iris-setosa']


#### k-NN Algorithm

Since Iris is a classic dataset, we're going to use a classic classification algorithm. You'll probably see this example a lot in books and online, it's *classifying the Iris dataset with **k nearest neighbors***. In kNN, we put all of our predictors in a *feature space*, stored with their classifications. Then when a new observation needs to be classified, we compare it to the nearest **k** observations and give it the classification of the majority nearest **k** observations.

In the diagram below, you can see that the predicted class for the star can change depending on the number of neighbors used.

![image](http://bdewilde.github.io/assets/images/2012-10-26-knn-concept.png)

*Food for thought*: kNN can also be used for regression problems! In a regression context, we predict based on the mean target of the k neighbors. For a more detailed explanation on kNN classification, [click here](https://saravananthirumuruganathan.wordpress.com/2010/05/17/a-detailed-introduction-to-k-nearest-neighbor-knn-algorithm/). Now for the code!


In [8]:
# KNN object and fit
knn = KNeighborsClassifier()
knn.fit(X=predictors,
        y=variable)

# Make predictions on test set
testPredictors = test.ix[:, :-1]
actual = test.ix[:, -1]
predictions = knn.predict(testPredictors)

# Merge the predicted and actual classifications
predictionsDF = pd.DataFrame(predictions, columns=['prediction'])
results = pd.concat([test.reset_index(drop=True), predictionsDF], axis=1)

# Model evaluation
matrix = metrics.confusion_matrix(y_true=results['class'],
                                  y_pred=results['prediction'])

print('------------------ Confusion Matrix ------------------')
print(matrix)

incorrect = results.ix[results['class'] != results['prediction'], ['class', 'prediction']]
if incorrect.shape[0] == 0:
    print('No incorrect classifications!')
else:
    print('-------------- Incorrect Classifications --------------')
    print(incorrect)

------------------ Confusion Matrix ------------------
[[18  0  0]
 [ 0  9  1]
 [ 0  0 17]]
-------------- Incorrect Classifications --------------
             class      prediction
0  Iris-versicolor  Iris-virginica


### Unsupervised Learning

Supervised learning is fairly straightforward, as we saw in the example. However, unsupervised learning requires some abstraction. Essentially, in an unsupervised exercise we are trying to either uncover hidden structure, find similarities, or reduce dimensionality in the data.

A common unsupervised example is clustering. If I were to hand you a bucket of rocks and ask you to put them into groups you may look at features like weight, volume, color and texture. Then you can group them by those characteristics. 

In a machine learning context, we can use unsupervised clustering as a pre-processing step. After our observations are clustered, we can easily add their cluster numbers as a column of predictors in the training data. Now we can use that new variable as a predictor in a regression or classification problem!

Eventually you may come across the problem of having *too many predictors*. If you have a set of 10 million observations and 200 predictors, then building a model on all predictors will be very expensive and time consuming. Thankfully, using techniques like [Principle Component Analysis](http://colah.github.io/posts/2014-10-Visualizing-MNIST/), we can reduce the number of predictors to only those with a *high variability* and save time in the long run.



#### Dimensionality Reduction using PCA

We're going to exemplify unsupervised learning with another classic dataset: the MNIST collection of handwritten digits. Each digit is a 28x28 pixel image, for 784 total pixels... 

\begin{matrix}
000 & 001 & 002 & 003 & ... & 026 & 027 \\
028 & 029 & 030 & 031 & ... & 054 & 055 \\
056 & 057 & 058 & 059 & ... & 082 & 083 \\
 \vdots &  \vdots &  \vdots &  \vdots & ... & \vdots & \vdots \\
728 & 729 & 730 & 731 & ... & 754 & 755 \\
756 & 757 & 758 & 759 & ... & 782 & 783 \\
\end{matrix}

The .csv contains the level of darkness for each pixel, organized as a table with 785 columns and 42,000 rows. This translates to the 784 pixel values *plus* the actual digit labels, and 42,000 digits...

\begin{matrix}
'1' & 001 & 002 & 003 & 004 & ... & 783 & 784 \\
'7' & 001 & 002 & 003 & 004 & ... & 783 & 784 \\
'3' & 001 & 002 & 003 & 004 & ... & 783 & 784 \\
 \vdots & \vdots &  \vdots &  \vdots &  \vdots & ... & \vdots & \vdots \\
'8' & 001 & 002 & 003 & 004 & ... & 783 & 784 \\
'6' & 001 & 002 & 003 & 004 & ... & 783 & 784 \\
\end{matrix}
 

In [10]:
# Read data from the repo you downloaded
location = os.path.realpath(os.path.join(os.getcwd(), "digits.csv"))
digits = pd.read_csv(location)

# Let's peak at the data
print("N rows =", digits.shape[0], '\n', 'N cols =', digits.shape[1])
print('------------------ Head of Data ------------------')
print(digits.head())

N rows = 42000 
 N cols = 785
------------------ Head of Data ------------------
   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  

In [6]:
from sklearn.decomposition import PCA

trainDigits, testDigits = cross_validation.train_test_split(digits,
                                                            test_size=0.3,
                                                            random_state=seed)
# Coerce training data to np arrays
trainLabel = trainDigits['label']
trainPixels = trainDigits.ix[:, 1:]

# Coerce testing data to np arrays
testLabel = testDigits['label']
testPixels = testDigits.ix[:, 1:]

# Fit a PCA model
pca = PCA(n_components = 10)
pca.fit(X=trainPixels)

PCA(copy=True, n_components=10, whiten=False)

In [11]:
# Let's train a KMeans model with the 10 principle components.
# First we need to transform the training set using the PCA parameters...
trainPCAPixels = pca.transform(trainPixels)

PCAdigitsKNN = KNeighborsClassifier()
PCAdigitsKNN.fit(X=trainPCAPixels,
              y=trainLabel)

# And now make predictions on the test set.
# First we need to transform the test set using the PCA parameters...
testPCAPixels = pca.transform(testPixels)

PCAdigitPredictions = PCAdigitsKNN.predict(testPCAPixels)
PCAdigitPredictionsDF = pd.DataFrame(PCAdigitPredictions, columns=['prediction'])
PCAdigitComparison = pd.concat([testLabel.reset_index(drop=True), PCAdigitPredictionsDF], axis=1)
PCAdigitsIncorrect = results.ix[PCAdigitComparison['label'] != \
                             PCAdigitComparison['prediction'], ['class', 'prediction']].shape[0]

PCAdigitsCorrect = PCAdigitComparison.shape[0] - PCAdigitsIncorrect
print('Digits correctly classified =', PCAdigitsCorrect )
print('Digits incorrectly classified =', PCAdigitsIncorrect)
print('Raw accuracy =', round(PCAdigitsCorrect / float(PCAdigitsCorrect + PCAdigitsIncorrect), 4))

NameError: name 'pca' is not defined

Now let's compare our accuracy\* of 99.98% to a baseline kNN using the full set of 784 features. Since we have over 70 times the predictors, the computation takes quite a while. For that reason, I downsampled the training data. If you want to see a *true* comparison, then comment out the sampling and *get ready to wait.*

\*Accuracy is a poor metric to evaluate classifiers. If we're classifying people as sick or healthy, and 99% of our population is healthy... then we could achieve 99% accuracy by simply saying that every person will be healthy! Check out [sensitivity and specificity](https://en.wikipedia.org/wiki/Sensitivity_and_specificity) for *binary* classifiers and [area under the curve](http://gim.unmc.edu/dxtests/roc3.htm) for *all* classifiers.

In [ ]:
# This cell takes a while to run. If you want to see the results, delete the block comment lines (""").

"""
digits2 = trainDigits.sample(frac=0.5,
                                random_state=seed)

trainDigits2, testDigits2= cross_validation.train_test_split(digits2,
                                                            test_size=0.3,
                                                            random_state=seed)

# Get the pixels and labels from train data
trainLabels2 = trainDigits2['label']
trainPixels2 = trainDigits2.ix[:, 1:]

# And the same from the testing data
testLabels2 = testDigits2['label']
testPixels2 = testDigits2.ix[:, 1:]

digitsKNN = KNeighborsClassifier()
digitsKNN.fit(X=trainPixels2,
             y=trainLabels2)

digitPredictions = digitsKNN.predict(testPixels2)

digitPredictionsDF = pd.DataFrame(digitPredictions, columns=['prediction'])
digitComparison = pd.concat([testLabels2.reset_index(drop=True), digitPredictionsDF], axis=1)
digitsIncorrect = results.ix[digitComparison['label'] != \
                             digitComparison['prediction'], ['class', 'prediction']].shape[0]

digitsCorrect = digitComparison.shape[0] - digitsIncorrect
print 'Digits correctly classified =', digitsCorrect 
print 'Digits incorrectly classified =', digitsIncorrect
print 'Raw accuracy =', round(digitsCorrect / float(digitsCorrect + digitsIncorrect), 4)
"""

### Your turn!

Alrighty! With the class time left you're going to load **and clean** the UCI dataset containing hourly measurements of road-level air quality in a busy Italian city. Then build a multiple linear regression model to predict Carbon Monoxidet from the predictors. 

Consult the [SciKit Learn docs](http://scikit-learn.org/stable/modules/classes.html) and the earlier code to get help. You can also ask Jason, but give it a solid attempt before asking.

In [1]:
# Load the dataset, and pring the head.
# Then go back, uncomment the line to remove the empty columns

location2 = os.path.realpath(os.path.join(os.getcwd(), "AirQualityUCI.csv"))
airQuality = pd.read_csv(location2, delimiter = ';', decimal = ',')

airQuality.drop(airQuality[[15,16]], axis=1, inplace=True)

print(airQuality.head(10))

assert airQuality.shape[1] == 15

NameError: name 'os' is not defined

In [12]:
# The air sensor sometime failed, and mising values were replaced with -200. So let's replace
# that with NumPy's NaN value.

# Hint for the first <FILL IN>: the values can happen in any row or column.
airQuality[airQuality.ix[:,:] == -200] = np.nan

print('N rows =', airQuality.shape[0], '\n', 'N cols =', airQuality.shape[1])
print(airQuality.isnull().sum())

NameError: name 'airQuality' is not defined

In [10]:
# Holy shit! That hyrdocarbon sensor really sucks! 
# For the sake of time, let's just drop that column too and remove rows with any NaNs.

airQuality.drop('NMHC(GT)', axis=1, inplace=True)
airQuality.dropna(axis=0, how='any', inplace=True)

print(airQuality.shape)
print(airQuality.head())
print(airQuality.isnull().sum())

(6941, 14)
         Date      Time  CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)  \
0  10/03/2004  18.00.00     2.6       1360.0      11.9         1046.0   
1  10/03/2004  19.00.00     2.0       1292.0       9.4          955.0   
2  10/03/2004  20.00.00     2.2       1402.0       9.0          939.0   
3  10/03/2004  21.00.00     2.2       1376.0       9.2          948.0   
4  10/03/2004  22.00.00     1.6       1272.0       6.5          836.0   

   NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)     T    RH  \
0    166.0        1056.0    113.0        1692.0       1268.0  13.6  48.9   
1    103.0        1174.0     92.0        1559.0        972.0  13.3  47.7   
2    131.0        1140.0    114.0        1555.0       1074.0  11.9  54.0   
3    172.0        1092.0    122.0        1584.0       1203.0  11.0  60.0   
4    131.0        1205.0    116.0        1490.0       1110.0  11.2  59.6   

       AH  
0  0.7578  
1  0.7255  
2  0.7502  
3  0.7867  
4  0.7888  
Date             0
Ti

In [51]:
# Great, now our data is free of missing values and we still have 7,400 observations!
# Now let's fit a model!

airTrain, airTest = cross_validation.train_test_split(airQuality, 
                                                      test_size=0.3,
                                                      random_state=seed)
Ncolumns = airTrain.shape[1]
# Pull out the labels from the train and test sets
trainLabels = airTrain['CO(GT)']
testLabels = airTest['CO(GT)']

# Pull out the predictors from the train and test sets
# Do NOT select 'Date' as a pedictor
trainVariables = airTrain.drop(['Date', 'CO(GT)', 'Time'], axis = 1, inplace = False)
testVariables = airTest.drop(['Date', 'CO(GT)', 'Time'], axis = 1, inplace = False)

print(trainVariables.head())
print('------------------------------------------')
print(testVariables.head())

assert trainVariables.shape[1] == testVariables.shape[1] == 11

      PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  \
6968       1053.0       3.5          680.0    166.0         890.0    108.0   
229        1064.0       4.3          728.0    115.0        1161.0     91.0   
6645       1250.0      14.5         1133.0    508.0         629.0    168.0   
9199        963.0       2.9          646.0    117.0         834.0     83.0   
221        1215.0       8.3          912.0    127.0         948.0    109.0   

      PT08.S4(NO2)  PT08.S5(O3)     T    RH      AH  
6968        1166.0        807.0  11.3  80.1  1.0741  
229         1461.0        842.0  13.5  58.4  0.8960  
6645        1274.0       1463.0  17.5  30.8  0.6095  
9199        1181.0        846.0  14.7  66.4  1.1006  
221         1547.0        993.0  14.2  58.3  0.9380  
------------------------------------------
      PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  \
7144       1016.0       4.9          760.0    254.0         835.0    112.0   
7236    

In [58]:
# Modeling time!
from sklearn.linear_model import LinearRegression

linearModel = LinearRegression()
linearModel.fit(X = trainVariables,
                y = trainLabels)

# Now join the true and predicted values together
testPredictions = linearModel.predict(testVariables)
testPredictionsDF = pd.DataFrame(testPredictions, columns=['prediction'])
testComparison = pd.concat([testLabels.reset_index(drop=True), testPredictionsDF], axis=1)

Let's see how well we did! Since this was a regression problem, we'll use the Mean Squared Error (MSE) to evaluate the model.

$$ \begin{align}
 MSE & = \frac{1}{n} \sum_{i=1}^n (Y_i - \hat Y_i)^2 \\
 & = \frac{1}{n} \sum_{i=1}^n (f(y_i) - \hat f(y_i) )^2 
 \end{align}$$

In [70]:
from sklearn.metrics import mean_squared_error
import math
# Let's see how well it did. The function below takes two columns, one of true values and 
# another with the predicted.

error = mean_squared_error(testComparison[[0]], testComparison[[1]])
print('MSE =', round(error, 4))
print('Root MSE =', round(math.sqrt(error),4))

MSE = 0.1715
Root MSE = 0.4141


The RMSE is in the units of the dependent variable, so now we can say... "We expect our model to predict the concentration of Carbon Monoxide within 0.41 $ \frac{mg}{m^3} $

##### Thanks to...

- [The MNIST digits dataset from Kaggle](https://www.kaggle.com/c/digit-recognizer/data)
- [The UCI Iris dataset](https://archive.ics.uci.edu/ml/datasets/Iris)
- [The UCI Air Quality dataset](https://archive.ics.uci.edu/ml/datasets/Air+Quality)
- [This **great** blog post on PCA](http://colah.github.io/posts/2014-10-Visualizing-MNIST/)
- [This Git Blog](http://bdewilde.github.io/blog/blogger/2012/10/26/classification-of-hand-written-digits-3/) for the kNN illustration